In [3]:
import torch
import numpy as np


import os
import glob
import re
import pandas


/tmp/ipykernel_208863/2016042400.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [4]:
prefix = '/mnt/hwfile/share_data/suzhongling/Ai4S/results/2T/open_CALcomplexHalf_COMint4kernel_TUNE1/Nodes176/ntask87/'
data2 = torch.cat([torch.load(prefix + f'/rank{rank}.pt') for rank in range(64)])

In [5]:
data2.shape

torch.Size([2999998, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
bitstringopen = torch.load('/mnt/petrelfs/suzhongling/Ai4S/TensorNetwork/sc38_reproduce_scheme_n53_m20_ABCDCDAB_3000000_einsum_10_open.pt')[-1]

In [9]:
len(bitstringopen)

2999998

In [10]:
permute_idx = [0, 2, 3, 8, 9, 4, 5, 6, 1, 7] # this is the permutation index of open result
open_qubits = sorted([22, 24, 31, 32, 38, 39, 40, 42, 44, 46])

In [11]:
close_qubits = list(set(range(53)) - set(open_qubits))

In [12]:
def find_corresponding_indices(dig, dig_raw_pre):
    mapping = {}
    for index, num in enumerate(dig):
        mapping[num] = index

    result = []
    for num in dig_raw_pre:
        assert num in mapping
        result.append(mapping[num])

    return result

In [13]:
import pandas as pd
import numpy as np

# 读取文件，设定分隔符为 "\t"
df = pd.read_csv("amps3M_all.txt", sep="\t", header=None)

def parse_complex(string):
    string = string.strip('()')  # remove parentheses
    real, imag = map(float, string.split(','))  # split on comma and convert to float
    return complex(real, imag)  # return complex number

# 将第二列的字符串转换成复数
df[1] = df[1].apply(lambda x: parse_complex(x))

# 为列命名
df.columns = ["Binary", "Complex", "Float"]
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'amps3M_all.txt'

In [10]:
df["BinaryClose"] = df["Binary"].apply(lambda x: "".join(np.array(list(x))[close_qubits]))

In [11]:
df["Closeidx"] = df["Binary"].apply(lambda x: 
                                   np.frombuffer(bytes(x,'ascii'), dtype=np.uint8)[open_qubits][permute_idx]
                                             - ord('0'))

In [12]:
corresponding_indices = find_corresponding_indices(bitstringopen,df["BinaryClose"])

In [13]:
len(corresponding_indices)

3000000

In [14]:
amp_exact = df["Complex"].values

In [15]:
amp_exact

array([ 9.34221e-09-4.77717e-10j, -3.58266e-10+3.27837e-09j,
        1.34830e-08+7.41322e-09j, ..., -5.19345e-09-3.29486e-10j,
       -1.46572e-08+1.16419e-08j,  4.50606e-09+4.84449e-09j])

In [16]:
x=0
bitstringopen[x], df["Closeidx"].values[x]

('0000000000000100000011010010000001110100011',
 array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [18]:
index_array = np.array(corresponding_indices).reshape(-1)
closeidx = (np.stack(df["Closeidx"].values)*np.array([2**(len(open_qubits)-x-1) for x in range(len(open_qubits))])).sum(axis=-1)
index_array = index_array*(2**len(open_qubits)) + closeidx

In [19]:
index_array

array([      2880,       3118,       2022, ..., 3071994936, 3071996351,
       3071989984])

In [20]:
data2.flatten().shape

torch.Size([3071997952])

In [27]:
amp_sel = data2.flatten().index_select(0, torch.from_numpy(index_array))

In [28]:
amplitude_exact = torch.tensor(amp_exact.copy(),dtype=torch.complex64)[:]
amplitude_appro = amp_sel.clone().to(torch.complex64)[:]
fidelity = (
    (amplitude_exact.conj() @ amplitude_appro.reshape(-1)).abs() /
    (amplitude_exact.abs().square().sum().sqrt() * amplitude_appro.abs().square().sum().sqrt())
).square().item()
fidelity

0.0002958333061542362

In [101]:
10/2**18

3.814697265625e-05

In [57]:
amp_exact[::-1]

array([ 4.50606e-09+4.84449e-09j, -1.46572e-08+1.16419e-08j,
       -5.19345e-09-3.29486e-10j, ...,  1.34830e-08+7.41322e-09j,
       -3.58266e-10+3.27837e-09j,  9.34221e-09-4.77717e-10j])

In [30]:
fidelity*np.log(1024)

0.0020505602207653594